In [1]:
import pandas as pd
import geopandas as gpd
import re
import os
from pathlib import Path
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

load_dotenv()

True

## Exploring feasibility of layering parks properties dataset into db-checkbook

### setup

In [2]:
csdb_df = pd.read_csv('../.output/historical_spend.csv')
csdb_gdf = gpd.GeoDataFrame(csdb_df)
parks_df = pd.read_csv('Parks_Properties.csv')
parks_gdf = gpd.GeoDataFrame(parks_df)

In [3]:
SQL_DIR = "../sql/"
DB_URL = "sqlite:///checkbook.db"
ENGINE = create_engine(DB_URL)

### exploration of layering in parks properties as source of geometries

files to reference from `db-cpdb`: 
- [cpdb_build_master.sql](../../db-cpdb/sql/cpdb_build_master.sql)
- [/workspace/products/db-cpdb/sql/attributes_dpr_fmsid.sql](../../db-cpdb/sql/attributes_dpr_string_id.sql)
- [/workspace/products/db-cpdb/sql/attributes_dpr_string_name.sql](../../db-cpdb/sql/attributes_dpr_string_name.sql)

---
Noticed that park names were being categorized as fixed assets *LAST* in the categorization query, which shouldn't be the case. This shouldn't affect % mapped but I updated the query to assign parks to fixed assets before moving onto the other categorties (lump sum, vehicles)

BEFORE updating priority of DPR query in `db-checkbook/sql/categorization.sql`:
- 8198 fixed asset projects unmapped in historical spend output 
- 930 of which mention DPR as an agency
---

running notes:
- where does `dpr_capitalprojects` come from? 
- we are interested in mapping projects that were not mapped in the first pass of assigning geometries from CPDB. that means projects that didn't have a matching FMS_ID in CPDB versions 2017-present. however, lots of unmapped projects clearly reference park names in their contract purpose/budget codes.
- Why would historical CPDB not include parks? Some parks may not have been part of an active capital project since pre-2017, which is the furthest back CPDB goes, so parks projects 2010-2017 are in limbo in the current CSDB output.
- best bet would be to do partial string matching between names cols ['EAPPLY', 'ADDRESS', 'NAME311', 'SIGNNAME'] col in Parks Properties and BC/CP in CSDB
    - this won't account for typos, differences in naming conventions ('i.e. 'Randall's Island' vs. 'Randall's Island Park') but it's a good first step 
    - if this is mildly successful and quick, continue to enhance
    - genuine fuzzy string matching is too computationally expensive

how to assess if this worked? 
- compare number of projects with geoms before and after 


In [4]:
# constants
parks_name_cols = ['EAPPLY', 'ADDRESS', 'NAME311', 'SIGNNAME']
gdf_nogeom = csdb_gdf[csdb_gdf['has_geometry']==False]
gdf = gdf_nogeom[gdf_nogeom['final_category']=='Fixed Asset'] # focus on unmapped fixed assets

print(parks_gdf.shape)
print(parks_gdf.columns)
print(gdf.shape)
print(gdf.columns)

(2044, 35)
Index(['ACQUISITIONDATE', 'ACRES', 'ADDRESS', 'BOROUGH', 'CLASS',
       'COMMUNITYBOARD', 'COUNCILDISTRICT', 'DEPARTMENT', 'EAPPLY', 'GISOBJID',
       'GISPROPNUM', 'GlobalID', 'JURISDICTION', 'LOCATION', 'MAPPED',
       'NAME311', 'NYS_ASSEMBLY', 'NYS_SENATE', 'OBJECTID', 'OMPPROPID',
       'PARENTID', 'PERMIT', 'PERMITDISTRICT', 'PERMITPARENT', 'PIP_RATABLE',
       'PRECINCT', 'RETIRED', 'SIGNNAME', 'SUBCATEGORY', 'TYPECATEGORY', 'URL',
       'US_CONGRESS', 'WATERFRONT', 'ZIPCODE', 'multipolygon'],
      dtype='object')
(8198, 23)
Index(['Unnamed: 0', 'fms_id', 'check_amount', 'contract_purpose',
       'budget_code', 'agency', 'bc_category', 'cp_category', 'cpdb_category',
       'ccpversion', 'maprojid', 'magency', 'magencyacr', 'projectid',
       'descriptio', 'geomsource', 'dataname', 'datasource', 'datadate',
       'geometry', 'cartodb_id', 'has_geometry', 'final_category'],
      dtype='object')


### what unmapped capital projects are from dpr?

In [5]:
gdf_dpr = gdf[gdf['agency'].str.contains('Department of Parks and Recreation')]

### what unmapped projects mention park keywords?

In [6]:
parksearch = ['PARK', 'PLAYGROUND', 'GARDEN']

temp = gdf.dropna(subset=['budget_code', 'contract_purpose'])
bc_park = temp[temp['budget_code'].str.contains('|'.join(parksearch))]['budget_code']
cp_park = temp[temp['contract_purpose'].str.contains('|'.join(parksearch))]['contract_purpose']
print(bc_park.shape)
print(cp_park.shape)

(591,)
(230,)


## layering in parks properties geometries using sqlalchemy

In [8]:
with ENGINE.connect() as conn:
    test_gdf = gdf_dpr.head(5)
    test_gdf.to_sql("csdb", ENGINE, if_exists="replace", index=False)
    # csdb_gdf.to_sql("csdb", ENGINE, if_exists="replace", index=False)
    # parks_gdf.to_sql("parks", ENGINE, if_exists="replace", index=False)
    # with open(SQL_DIR + "parks.sql", "r") as query_file:
    #     query = query_file.read()
    # conn.execute(text(query))

    # csdb_with_parks = pd.read_sql_table("csdb", conn)
   

In [ ]:
csdb_with_parks[csdb_with_parks['geometry'].isnull()].shape

591 capital projects categorized as Fixed Asset and not yet assigned a geometry have budget codes containing words relating to parks
230 capital project with the same criteria have contract purposes containing words relating to parks
There is some overlap between the two but this is promising regardless, given that we have only mapped 3880 capital projects so far

*Idea:* filter unmapped capital projects to be as small a subset as possible, then apply fuzzy string mtching to get park name or address out of budget code / contract purpose